In [ ]:
#설치 파일
pip install mlxtend
pip install plotly
pip install xgboost
pip install lightgbm
pip install catboost

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [7]:
import matplotlib.pyplot as plt
import warnings
import seaborn as sns
from colorama import Fore, Back, Style 
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score
from mlxtend.plotting import plot_confusion_matrix
from plotly.offline import plot, iplot, init_notebook_mode
import plotly.graph_objs as go
from plotly.subplots import make_subplots
import plotly.express as px
from statsmodels.formula.api import ols
import plotly.graph_objs as gobj

init_notebook_mode(connected=True)
warnings.filterwarnings("ignore")
import plotly.figure_factory as ff

%matplotlib inline

import xgboost
import lightgbm
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from catboost import CatBoostClassifier

In [8]:
data = pd.read_csv('datasets\HF_data.csv')
data.head()

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time,DEATH_EVENT
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4,1
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6,1
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7,1
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7,1
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8,1


In [11]:
dia_data = [data["diabetes"].values]
dia_labels = ['diabetes']

fig = ff.create_distplot(dia_data, dia_labels)
fig.update_layout(title_text='diabetes plot')

fig.show()

In [21]:
#성별 관련 없이 당뇨 발생시 사망여부 분석

yes_dia = data[data["diabetes"] == 1] #yes
no_dia = data[data["diabetes"] == 0] #no

dia_survi = yes_dia[data["DEATH_EVENT"]==0]
dia_death = yes_dia[data["DEATH_EVENT"]==1] 
nodia_survi = no_dia[data["DEATH_EVENT"]==0]
nodia_death = no_dia[data["DEATH_EVENT"]==1] 

labels = ['dia - Survived','dia - Death', "No dia -  Survived", "No dia - Death"]
values = [len(yes_dia[data["DEATH_EVENT"]==0]),len(yes_dia[data["DEATH_EVENT"]==1]),
         len(no_dia[data["DEATH_EVENT"]==0]),len(no_dia[data["DEATH_EVENT"]==1])]

fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.4)])
fig.update_layout(
    title_text="Analysis on Survival - diabetes")

fig.show()

300개의 데이터 중 
당뇨 존재 : 41.8%
당뇨 X 사망 : 18.7%
당뇨 O 사망 : 13.4%

In [23]:
#당뇨 발생시 사망률

yes_dia = data[data["diabetes"] == 1] #yes

dia_survi = yes_dia[data["DEATH_EVENT"]==0]
dia_death = yes_dia[data["DEATH_EVENT"]==1] 

labels = ['dia - Survived','dia - Death']
values = [len(yes_dia[data["DEATH_EVENT"]==0]),len(yes_dia[data["DEATH_EVENT"]==1])]

fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.4)])
fig.update_layout(
    title_text="Analysis on Survival - diabetes")

fig.show()

당뇨 발생시 생존률 68% , 사망률 32%

In [25]:
#성별 당뇨 환자 분석

male = data[data["sex"] == 1] 
female = data[data["sex"] == 0]

male_dia = male[data["diabetes"] == 1] #남성 당뇨
male_dia = male[data["diabetes"] == 0]
female_dia = female[data["diabetes"] == 1] #여성 당뇨
female_dia = female[data["diabetes"] == 0]

labels = ['Male - no dia','Male - dia', "Female - no dia", "Female - dia"]
values = [len(male[data["diabetes"] == 0]),len(male[data["diabetes"] == 1]),
         len(female[data["diabetes"] == 0]),len(female[data["diabetes"] == 1])]

fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.4)])
fig.update_layout(
    title_text="Sex - diabetes")

fig.show()

남성 당뇨 환자 : 23.4%
여성 당뇨 환자 : 18.4%

In [24]:
#성별 포함 당뇨 발생시 사망여부 분석

male = data[data["sex"] == 1] 
female = data[data["sex"] == 0]

male_dia = male[data["diabetes"] == 1] #남성 당뇨
female_dia = female[data["diabetes"] == 1] #여성 당뇨

male_dia_survi = male_dia[data["DEATH_EVENT"]==0]
male_dia_death = male_dia[data["DEATH_EVENT"]==1] 
female_dia_survi = female_dia[data["DEATH_EVENT"]==0]
female_dia_death = female_dia[data["DEATH_EVENT"]==1] 

labels = ['Male dia - Survived','Male dia - Death', "Female dia -  Survived", "Female dia - Death"]
values = [len(male_dia[data["DEATH_EVENT"]==0]),len(male_dia[data["DEATH_EVENT"]==1]),
         len(female_dia[data["DEATH_EVENT"]==0]),len(female_dia[data["DEATH_EVENT"]==1])]

fig = go.Figure(data=[go.Pie(labels=labels, values=values, hole=.4)])
fig.update_layout(
    title_text="Sex - diabetes Analysis")

fig.show()

남성 당뇨 환자 사망률 : 16%
여성 당뇨 환자 사망률 : 16%

In [32]:
train_set = data.iloc[:, :-1]
label_set = data.iloc[:, -1]

train_set

,age,anaemia,creatinine_phosphokinase,diabetes,ejection_fraction,high_blood_pressure,platelets,serum_creatinine,serum_sodium,sex,smoking,time
0,75.0,0,582,0,20,1,265000.00,1.9,130,1,0,4
1,55.0,0,7861,0,38,0,263358.03,1.1,136,1,0,6
2,65.0,0,146,0,20,0,162000.00,1.3,129,1,1,7
3,50.0,1,111,0,20,0,210000.00,1.9,137,1,0,7
4,65.0,1,160,1,20,0,327000.00,2.7,116,0,0,8
...,...,...,...,...,...,...,...,...,...,...,...,...
294,62.0,0,61,1,38,1,155000.00,1.1,143,1,1,270
295,55.0,0,1820,0,38,0,270000.00,1.2,139,0,0,271
296,45.0,0,2060,1,60,0,742000.00,0.8,138,0,0,278
297,45.0,0,2413,0,38,0,140000.00,1.4,140,1,1,280


In [34]:
# 사망 확률 구하기(회귀 - 선형 회귀 사용)
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score

lin_reg = LinearRegression()
lin_reg.fit(train_set, label_set)

lin_scores = cross_val_score(lin_reg, train_set, label_set, cv=3, scoring="neg_root_mean_squared_error")
lin_rmse_scores = np.sqrt(-lin_scores)

def display_scores(scores):
    print("Scores:", scores)
    print("Mean:", scores.mean())
    print("Standard deviation:", scores.std())

display_scores(lin_rmse_scores)

Scores: [0.72278908 0.66575749 0.58013718]
Mean: 0.6562279173215066
Standard deviation: 0.05862593925208004


확률 검증 모델 : 3-fold 교차 검증시 평균 0.05의 오차를 보임